# Convolutional Neural Network

The objective of this file is to provide work for the implmentation and execution of the convolutional neural.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score

In [2]:
# Hardcode which dataset for 1D CNN {Cho = 1, Iyer = 2}
option = 2

n_features = 16
n_outputs  = 5

# locate Cho data
if(option == 1):
    datapath = '../data/'
    filename = 'cho.txt'
# Locate Iyer data
else:
    datapath = '../data/'
    filename = 'iyer.txt'
    n_features = 12
    n_outputs  = 10

# Load data as a numpy array
dataSet = np.genfromtxt(datapath+filename)

# Remove -1 extraenous class from dataset
if(option != 1):
    dataSet = dataSet[dataSet[:,1]>-1, :]


# Extract features from dataSet
X = dataSet [:,2:]

# Extract class labels (for hyperparamter tuning and evaluation)
y = dataSet[:,1]

# Create simple test, train split for demonstration (Tensorflow)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.67, 
                                                    random_state=42)
# Normalize data
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X = X.reshape((X.shape[0], X.shape[1],1))
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],1))

print(X.shape)

(484, 12, 1)


In [66]:
# Model must be defined for Grid Search CV
def create_model(filter_num=0, ker_size=0, dropout_r=0.0,dense=0):

    model = Sequential()
    model.add(Conv1D(filters=filter_num, kernel_size=ker_size, activation='relu', input_shape=(n_features, 1)))
    model.add(Dropout(dropout_r))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(dense, activation='relu'))
    model.add(Dense(n_outputs+1, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [32]:
# Build model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size = 10, verbose=0)

# Create range of values for Grid Search CV
filter_num = [8, 16,32]
ker_size    = [2, 4, 6]
dropout_r  = [0.5,0.6,0.7]
dense      = [20,60,120]

# Generate dictionary for matching model parameters
param_grid = dict(filter_num=filter_num, ker_size=ker_size, dropout_r=dropout_r, dense=dense)

# Define the K-fold crossvalidation method
cv_method = KFold(n_splits=10, shuffle=True)

# Perform Hyperparameterization function GridSearchCV(...)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv_method, scoring='accuracy',verbose=10)

# Extract results
grid.fit(X,y)


Fitting 10 folds for each of 81 candidates, totalling 810 fits
Fitting 10 folds for each of 81 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  61 tasks   

KeyboardInterrupt: 

KeyboardInterrupt: 

In [28]:
# Print grid results and display best parameters
print(grid.cv_results_)
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))


{'mean_fit_time': array([13.64881208, 12.48634889, 12.23947093, 13.26500795, 13.56234176,
       14.3876826 , 15.07664008, 14.57159564, 14.60995555, 13.24105809,
       13.65430124, 14.54712236, 15.23403192, 15.26401289, 15.28186743,
       14.04450095, 14.04808283, 14.51010077, 13.27183242, 13.61537361,
       12.95290933, 13.75034921, 14.86689181, 15.46683431, 16.2215317 ,
       16.15559082, 15.7871891 , 13.60152972, 13.41840022, 13.44074409,
       13.98856981, 13.6584491 , 13.26404049, 15.36632688, 17.78952665,
       15.79327133, 14.37996325, 14.2809938 , 13.74946322, 14.72259164,
       16.52140989, 15.37126265, 15.24040978, 17.14210916, 18.04217799,
       16.89437177, 14.9286025 , 16.09355967, 16.38890049, 15.45350072,
       15.31856949, 17.0979141 , 16.54405375, 17.48659863, 13.90942688,
       13.78001366, 13.76961832, 15.85111442, 15.70735216, 15.09014435,
       15.67900784, 16.06034417, 15.68788519, 13.78811107, 13.9513818 ,
       14.2116045 , 15.12751834, 14.9716588 , 

{'mean_fit_time': array([13.64881208, 12.48634889, 12.23947093, 13.26500795, 13.56234176,
       14.3876826 , 15.07664008, 14.57159564, 14.60995555, 13.24105809,
       13.65430124, 14.54712236, 15.23403192, 15.26401289, 15.28186743,
       14.04450095, 14.04808283, 14.51010077, 13.27183242, 13.61537361,
       12.95290933, 13.75034921, 14.86689181, 15.46683431, 16.2215317 ,
       16.15559082, 15.7871891 , 13.60152972, 13.41840022, 13.44074409,
       13.98856981, 13.6584491 , 13.26404049, 15.36632688, 17.78952665,
       15.79327133, 14.37996325, 14.2809938 , 13.74946322, 14.72259164,
       16.52140989, 15.37126265, 15.24040978, 17.14210916, 18.04217799,
       16.89437177, 14.9286025 , 16.09355967, 16.38890049, 15.45350072,
       15.31856949, 17.0979141 , 16.54405375, 17.48659863, 13.90942688,
       13.78001366, 13.76961832, 15.85111442, 15.70735216, 15.09014435,
       15.67900784, 16.06034417, 15.68788519, 13.78811107, 13.9513818 ,
       14.2116045 , 15.12751834, 14.9716588 , 

In [11]:
%reload_ext tensorboard

In [3]:
import tensorflow as tf
import datetime, os

In [73]:
!rm -rf ./logs/

In [67]:
# Build model for Tensorflow to display Accuracy Vs. Epoch and Loss Vs. Epoch
def create_model_tf():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=16, kernel_size=6, activation='relu', input_shape=(n_features, 1)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(n_outputs+1, activation='softmax')
  ])

In [74]:
def train_model():
    
  """
    Best hyper parameters:
    
    'dense': 20, 
    'dropout_r': 0.5, 
    'filter_num': 16, 
    'ker_size': 6 
    
    with a score of 0.78
  """    

  # Regenerate model with best parameters
  model = create_model_tf()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  # Prepare results for Tensorflow Module (Platform)
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  # Perform traning and rough validation
  model.fit(x=X_train, 
            y=y_train, 
            epochs=100, 
            validation_data=(X_test, y_test), 
            callbacks=[tensorboard_callback])

  # Return predicted y labels
  y_hat = model.predict_classes(X_test, verbose=0)

  def multiclass_roc_auc_score(y_test, y_pred, average="macro"): # Method from https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)
  def multiclass_f_score(y_test, y_pred, average="macro"): # Method from https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return f1_score(y_test, y_pred, average=average)

  # Display f1 score and AUC score
  print(multiclass_f_score(y_test,y_hat))
  print(multiclass_roc_auc_score(y_test,y_hat))



train_model()

Epoch 1/100
11/11 [==============================] - 3s 256ms/step - loss: 2.4141 - accuracy: 0.2195 - val_loss: 2.4169 - val_accuracy: 0.2375
Epoch 2/100
11/11 [==============================] - 0s 5ms/step - loss: 2.3261 - accuracy: 0.3428 - val_loss: 2.3496 - val_accuracy: 0.2313
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 2.2629 - accuracy: 0.3386 - val_loss: 2.2888 - val_accuracy: 0.2438
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 2.1427 - accuracy: 0.3676 - val_loss: 2.2312 - val_accuracy: 0.2500
Epoch 5/100
11/11 [==============================] - 0s 5ms/step - loss: 2.0898 - accuracy: 0.3498 - val_loss: 2.1632 - val_accuracy: 0.3000
Epoch 6/100
11/11 [==============================] - 0s 5ms/step - loss: 2.0153 - accuracy: 0.3507 - val_loss: 2.0866 - val_accuracy: 0.3438
Epoch 7/100
11/11 [==============================] - 0s 5ms/step - loss: 1.8767 - accuracy: 0.4234 - val_loss: 2.0149 - val_accuracy: 0.3875
Epoch 8/100

Epoch 59/100
11/11 [==============================] - 0s 8ms/step - loss: 0.6784 - accuracy: 0.7409 - val_loss: 0.7549 - val_accuracy: 0.7750
Epoch 60/100
11/11 [==============================] - 0s 8ms/step - loss: 0.6467 - accuracy: 0.7482 - val_loss: 0.7521 - val_accuracy: 0.7500
Epoch 61/100
11/11 [==============================] - 0s 6ms/step - loss: 0.6203 - accuracy: 0.7814 - val_loss: 0.7459 - val_accuracy: 0.7375
Epoch 62/100
11/11 [==============================] - 0s 5ms/step - loss: 0.5861 - accuracy: 0.8060 - val_loss: 0.7452 - val_accuracy: 0.7188
Epoch 63/100
11/11 [==============================] - 0s 5ms/step - loss: 0.6373 - accuracy: 0.7526 - val_loss: 0.7306 - val_accuracy: 0.7625
Epoch 64/100
11/11 [==============================] - 0s 7ms/step - loss: 0.6713 - accuracy: 0.7441 - val_loss: 0.7388 - val_accuracy: 0.6875
Epoch 65/100
11/11 [==============================] - 0s 7ms/step - loss: 0.6494 - accuracy: 0.7620 - val_loss: 0.7395 - val_accuracy: 0.6812
Epoch 

C:\Users\TheAr\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [10]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 16600), started 1 day, 13:20:40 ago. (Use '!kill 16600' to kill it.)

In [46]:
# Code for 2D-CNN

PIE_datapath = '../data/PIE_32x32/'
PIE_test_filenames = ['StTestFile'+str(i)+'.txt' for i in range(1,11)]
PIE_train_filenames = ['StTrainFile'+str(i)+'.txt' for i in range(1,11)]


PIE_train_data_files = [np.genfromtxt(PIE_datapath+PIE_train_filenames[i],delimiter=' ') for i in range(0,2)]


rowData = []

for i in PIE_train_data_files:
    for row in i:
        rowData.append(row)
    
PIE_train_data = np.array(rowData)

PIE_Xtrain = PIE_train_data[:,0:1024]
PIE_ytrain = PIE_train_data[:,1024]

PIE_test_filename = PIE_test_filenames[0]
PIE_test_data = np.genfromtxt(PIE_datapath+PIE_test_filename,delimiter=' ')
PIE_Xtest = PIE_test_data[:,0:1024]
PIE_ytest = PIE_test_data[:,1024]


In [47]:

listofimages = []
for row in range(PIE_Xtrain.shape[0]):
  currimage = PIE_Xtrain[row,0:1024]
  currreshaped = np.reshape(currimage,(32,32,1))
  listofimages.append(currreshaped)

PIE_Xtrain = np.array(listofimages)

listofimages = []
for row in range(PIE_Xtest.shape[0]):
  currimage = PIE_Xtest[row,0:1024]
  currreshaped = np.reshape(currimage,(32,32,1))
  listofimages.append(currreshaped)

PIE_Xtest = np.array(listofimages)


In [48]:
print(PIE_Xtrain)
print(PIE_Xtrain.shape)

[[[[0.031373]
   [0.035294]
   [0.035294]
   ...
   [0.039216]
   [0.031373]
   [0.035294]]

  [[0.035294]
   [0.039216]
   [0.035294]
   ...
   [0.039216]
   [0.031373]
   [0.035294]]

  [[0.035294]
   [0.035294]
   [0.035294]
   ...
   [0.043137]
   [0.039216]
   [0.035294]]

  ...

  [[0.027451]
   [0.023529]
   [0.019608]
   ...
   [0.019608]
   [0.019608]
   [0.019608]]

  [[0.015686]
   [0.019608]
   [0.019608]
   ...
   [0.019608]
   [0.019608]
   [0.019608]]

  [[0.019608]
   [0.019608]
   [0.019608]
   ...
   [0.019608]
   [0.023529]
   [0.019608]]]


 [[[0.039216]
   [0.043137]
   [0.043137]
   ...
   [0.043137]
   [0.035294]
   [0.035294]]

  [[0.05098 ]
   [0.05098 ]
   [0.047059]
   ...
   [0.047059]
   [0.043137]
   [0.039216]]

  [[0.070588]
   [0.066667]
   [0.054902]
   ...
   [0.047059]
   [0.039216]
   [0.043137]]

  ...

  [[0.023529]
   [0.023529]
   [0.027451]
   ...
   [0.019608]
   [0.019608]
   [0.019608]]

  [[0.019608]
   [0.019608]
   [0.019608]
   ...
   [0

In [49]:

#model definition and building
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = (32, 32, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(69, activation="softmax"))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.summary()

model.fit(PIE_Xtrain, PIE_ytrain, batch_size=64, epochs=10, validation_data=(PIE_Xtest,PIE_ytest))

y_hat = np.argmax(model.predict(PIE_Xtest), axis=-1)

def multiclass_roc_auc_score(y_test, y_pred, average="macro"): # Method from https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
  lb = LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(y_test, y_pred, average=average)
def multiclass_f_score(y_test, y_pred, average="macro"): # Method from https://medium.com/@plog397/auc-roc-curve-scoring-function-for-multi-class-classification-9822871a6659
  lb = LabelBinarizer()
  lb.fit(y_test)
  y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return f1_score(y_test, y_pred, average=average)

print(multiclass_f_score(PIE_ytest,y_hat))
print(multiclass_roc_auc_score(PIE_ytest,y_hat))


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 69)               

In [36]:
def td_create_model(filter_num=0, ker_size=0, dropout_r=0.0,dense=0):

    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape = (32, 32, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(69, activation="softmax"))
      
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn=td_create_model, epochs=100, batch_size = 10, verbose=0)

filter_num = [8, 16,32]
ker_size    = [2, 4, 6]
dropout_r  = [0.5,0.6,0.7]
dense      = [20,60,120]


param_grid = dict(filter_num=filter_num, ker_size=ker_size, dropout_r=dropout_r, dense=dense)

cv_method = KFold(n_splits=5, shuffle=True)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv_method, scoring='accuracy',verbose=10)
grid.fit(X,y)
